In [ ]:
from dataclasses import dataclass
from datetime import datetime
from typing import List, Optional

@dataclass
class Product:
    id: str
    name: str
    price: float

@dataclass
class Warehouse:
    id: str
    name: str
    location: str

@dataclass
class Quantity:
    amount: int

@dataclass
class Inventory:
    product: Product
    warehouse: Warehouse
    quantity: Quantity
    last_updated: datetime

class InventoryService:
    def add_inventory(self, inventory: Inventory) -> None:
        inventory.quantity.amount += 1
        inventory.last_updated = datetime.now()

    def reduce_inventory(self, inventory: Inventory) -> None:
        inventory.quantity.amount -= 1
        inventory.last_updated = datetime.now()

    def change_location(self, inventory: Inventory, new_location: Warehouse) -> None:
        inventory.warehouse = new_location
        inventory.last_updated = datetime.now()

class ProductRepository:
    def __init__(self):
        self.products = []

    def save(self, product: Product) -> None:
        self.products.append(product)

    def get_by_id(self, id: str) -> Optional[Product]:
        for product in self.products:
            if product.id == id:
                return product
        return None

    def get_all(self) -> List[Product]:
        return self.products

class WarehouseRepository:
    def __init__(self):
        self.warehouses = []

    def save(self, warehouse: Warehouse) -> None:
        self.warehouses.append(warehouse)

    def get_by_id(self, id: str) -> Optional[Warehouse]:
        for warehouse in self.warehouses:
            if warehouse.id == id:
                return warehouse
        return None

    def get_all(self) -> List[Warehouse]:
        return self.warehouses

class InventoryRepository:
    def __init__(self):
        self.inventories = []

    def save(self, inventory: Inventory) -> None:
        self.inventories.append(inventory)

    def get_by_product_and_warehouse(self, product: Product, warehouse: Warehouse) -> Optional[Inventory]:
        for inventory in self.inventories:
            if inventory.product == product and inventory.warehouse == warehouse:
                return inventory
        return None

    def get_all(self) -> List[Inventory]:
        return self.inventories


In [ ]:
import unittest
from datetime import datetime

class TestDDD(unittest.TestCase):
    def setUp(self):
        self.product = Product(id="p1", name="Product1", price=100.0)
        self.warehouse = Warehouse(id="w1", name="Warehouse1", location="Location1")
        self.quantity = Quantity(amount=10)
        self.inventory = Inventory(product=self.product, warehouse=self.warehouse, quantity=self.quantity, last_updated=datetime.now())
        self.product_repo = ProductRepository()
        self.warehouse_repo = WarehouseRepository()
        self.inventory_repo = InventoryRepository()
        self.inventory_service = InventoryService()

    def test_product_repository(self):
        self.product_repo.save(self.product)
        self.assertEqual(self.product, self.product_repo.get_by_id("p1"))
        self.assertEqual([self.product], self.product_repo.get_all())

    def test_warehouse_repository(self):
        self.warehouse_repo.save(self.warehouse)
        self.assertEqual(self.warehouse, self.warehouse_repo.get_by_id("w1"))
        self.assertEqual([self.warehouse], self.warehouse_repo.get_all())

    def test_inventory_repository(self):
        self.inventory_repo.save(self.inventory)
        self.assertEqual(self.inventory, self.inventory_repo.get_by_product_and_warehouse(self.product, self.warehouse))
        self.assertEqual([self.inventory], self.inventory_repo.get_all())

    def test_inventory_service(self):
        self.inventory_service.add_inventory(self.inventory)
        self.assertEqual(11, self.inventory.quantity.amount)
        self.inventory_service.reduce_inventory(self.inventory)
        self.assertEqual(10, self.inventory.quantity.amount)
        new_warehouse = Warehouse(id="w2", name="Warehouse2", location="Location2")
        self.inventory_service.change_location(self.inventory, new_warehouse)
        self.assertEqual(new_warehouse, self.inventory.warehouse)




In [ ]:
unittest.main(argv=['first-arg-is-ignored'], exit=False, verbosity=2)

test_inventory_repository (__main__.TestDDD) ... ok
test_inventory_service (__main__.TestDDD) ... ok
test_product_repository (__main__.TestDDD) ... ok
test_warehouse_repository (__main__.TestDDD) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.026s

OK


In [ ]:
from sqlalchemy import Column, Integer, String, DateTime, ForeignKey
from sqlalchemy.orm import relationship,declarative_base
from datetime import datetime

Base = declarative_base()

class Product(Base):
    __tablename__ = 'products'
    id = Column('product_id', Integer, primary_key=True)
    name = Column(String(64))
    price = Column(Integer)

    def __init__(self, name, price):
        self.name = name
        self.price = price


class Warehouse(Base):
    __tablename__ = 'warehouses'
    id = Column('warehouse_id', Integer, primary_key=True)
    name = Column(String(64))
    location = Column(String(64))

    def __init__(self, name, location):
        self.name = name
        self.location = location


class Quantity(Base):
    __tablename__ = 'quantities'
    id = Column('quantity_id', Integer, primary_key=True)
    amount = Column(Integer)

    def __init__(self, amount):
        self.amount = amount


class Inventory(Base):
    __tablename__ = 'inventories'
    id = Column('inventory_id', Integer, primary_key=True)
    product_id = Column(Integer, ForeignKey('products.product_id'))
    product = relationship('Product')
    warehouse_id = Column(Integer, ForeignKey('warehouses.warehouse_id'))
    warehouse = relationship('Warehouse')
    quantity_id = Column(Integer, ForeignKey('quantities.quantity_id'))
    quantity = relationship('Quantity')
    last_updated = Column(DateTime, default=datetime.utcnow)

    def __init__(self, product, warehouse, quantity):
        self.product = product
        self.warehouse = warehouse
        self.quantity = quantity

    def update_time(self):
        self.last_updated = datetime.utcnow()


class InventoryService:
    def __init__(self, inventory_repository):
        self.inventory_repository = inventory_repository

    def add_inventory(self, inventory):
        self.inventory_repository.save(inventory)
        inventory.update_time()

    def reduce_inventory(self, inventory):
        inventory.quantity.amount -= 1
        self.inventory_repository.save(inventory)
        inventory.update_time()

    def change_location(self, inventory, new_location):
        inventory.warehouse = new_location
        self.inventory_repository.save(inventory)
        inventory.update_time()


In [ ]:
from abc import ABC, abstractmethod
from sqlalchemy.orm import Session

class AbstractRepository(ABC):
    def __init__(self, session: Session):
        self.session = session

    @abstractmethod
    def save(self, entity):
        pass

    @abstractmethod
    def get_by_id(self, id):
        pass

    @abstractmethod
    def get_all(self):
        pass


class ProductRepository(AbstractRepository):
    def save(self, product: Product):
        self.session.add(product)
        self.session.commit()

    def get_by_id(self, id: int):
        return self.session.get(Product, id)

    def get_all(self):
        return self.session.query(Product).all()


class WarehouseRepository(AbstractRepository):
    def save(self, warehouse: Warehouse):
        self.session.add(warehouse)
        self.session.commit()

    def get_by_id(self, id: int):
        return self.session.get(Warehouse, id)

    def get_all(self):
        return self.session.query(Warehouse).all()


class InventoryRepository(AbstractRepository):
    def save(self, inventory: Inventory):
        self.session.add(inventory)
        self.session.commit()

    def get_by_id(self, id: int):
        return self.session.query(Inventory).get(id)

    def get_by_product_and_warehouse(
        self, product: Product, warehouse: Warehouse):
        return self.session.query(Inventory).filter_by(
            product_id=product.id, warehouse_id=warehouse.id).first()

    def get_all(self):
        return self.session.query(Inventory).all()


In [ ]:
import unittest
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from datetime import datetime

class TestDDD(unittest.TestCase):
    def setUp(self):
        engine = create_engine('sqlite:///:memory:')
        Base.metadata.create_all(engine)
        Session = sessionmaker(bind=engine)
        self.session = Session()

        self.product_repository = ProductRepository(self.session)
        self.warehouse_repository = WarehouseRepository(self.session)
        self.inventory_repository = InventoryRepository(self.session)

        self.product = Product('product1', 1000)
        self.warehouse = Warehouse('warehouse1', 'location1')
        self.quantity = Quantity(10)
        self.inventory = Inventory(
            self.product, self.warehouse, self.quantity)

        self.product_repository.save(self.product)
        self.warehouse_repository.save(self.warehouse)
        self.inventory_repository.save(self.inventory)

        self.session.commit()

    def test_product_repository(self):
        retrieved_product = self.product_repository.get_by_id(self.product.id)
        self.assertEqual(retrieved_product.name, 'product1')
        self.assertEqual(retrieved_product.price, 1000)

    def test_warehouse_repository(self):
        retrieved_warehouse = self.warehouse_repository.get_by_id(
            self.warehouse.id)
        self.assertEqual(retrieved_warehouse.name, 'warehouse1')
        self.assertEqual(retrieved_warehouse.location, 'location1')

    def test_inventory_repository(self):
        retrieved_inventory = \
          self.inventory_repository.get_by_product_and_warehouse(
              self.product, self.warehouse)
        self.assertEqual(retrieved_inventory.product.name, 'product1')
        self.assertEqual(retrieved_inventory.warehouse.name, 'warehouse1')
        self.assertEqual(retrieved_inventory.quantity.amount, 10)

    def test_inventory_service(self):
        inventory_service = InventoryService(self.inventory_repository)

        # add inventory
        new_product = Product('product2', 2000)
        new_warehouse = Warehouse('warehouse2', 'location2')
        new_quantity = Quantity(5)
        new_inventory = Inventory(new_product, new_warehouse, new_quantity)
        inventory_service.add_inventory(new_inventory)
        self.session.commit()

        retrieved_inventory = \
          self.inventory_repository.get_by_product_and_warehouse(
              new_product, new_warehouse)
        self.assertEqual(retrieved_inventory.product.name, 'product2')
        self.assertEqual(retrieved_inventory.warehouse.name, 'warehouse2')
        self.assertEqual(retrieved_inventory.quantity.amount, 5)

        # reduce inventory
        inventory_service.reduce_inventory(retrieved_inventory)
        self.session.commit()

        retrieved_inventory = \
          self.inventory_repository.get_by_product_and_warehouse(
              new_product, new_warehouse)
        self.assertEqual(retrieved_inventory.quantity.amount, 4)

        # change location
        another_warehouse = Warehouse('warehouse3', 'location3')
        inventory_service.change_location(
            retrieved_inventory, another_warehouse)
        self.session.commit()

        retrieved_inventory = \
          self.inventory_repository.get_by_product_and_warehouse(
              new_product, another_warehouse)
        self.assertEqual(retrieved_inventory.warehouse.name, 'warehouse3')



In [ ]:
unittest.main(argv=['first-arg-is-ignored'], exit=False, verbosity=2)

test_inventory_repository (__main__.TestDDD) ... ok
test_inventory_service (__main__.TestDDD) ... ok
test_product_repository (__main__.TestDDD) ... ok
test_warehouse_repository (__main__.TestDDD) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.100s

OK
